In [17]:
import numpy as np 
import pandas as pd
import re
import string
import gensim
import pickle
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk import punkt
from nltk import wordnet

from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, GRU, Bidirectional, Conv1D, GlobalMaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
data_columns = ["target", "tweet", "username"]
data_encoding = "ISO-8859-1"
data=pd.read_csv('./TwitterDataset.csv',encoding = data_encoding, usecols=data_columns)

X = data.iloc[:, 1]
Y = data.iloc[:, 0]

print(X.head())
print(data['target'].value_counts())


0    the real reason why you're sad? you're attache...
1        my biggest problem is overthinking everything
2    the worst sadness is the sadness you've taught...
3    i cannot make you understand. i cannot make an...
4    i don't think anyone really understands how ti...
Name: tweet, dtype: object
0    4695
1    3440
Name: target, dtype: int64


In [3]:
def preprocess(tweet):
    username = "@\S+"
    new_tweet = re.sub(username, ' ',tweet) # Remove @tags
    
    new_tweet = new_tweet.lower() # Smart lowercase
    
    new_tweet = re.sub(r'\d+', ' ', new_tweet) # Remove numbers
    
    text_noise = "https?:\S+|http?:\S|[^A-Za-z0-9]+" 
    new_tweet = re.sub(text_noise, ' ', new_tweet) # Remove links
    
    new_tweet = new_tweet.translate(new_tweet.maketrans('','',string.punctuation)) # Remove Punctuation
    
    new_tweet = new_tweet.strip() # Remove white spaces
    
    new_tweet = word_tokenize(new_tweet) # Tokenize into words
    
    new_tweet = ' '.join([word for word in new_tweet if word.isalpha()]) # Remove non alphabetic tokens
    
    stop_words = set(stopwords.words('english'))
    new_tweet = ' '.join([word for word in new_tweet.split() if not word in stop_words]) # Filter out stop words
    
    lemmatizer = WordNetLemmatizer()
    new_tweet = ' '.join([lemmatizer.lemmatize(word,"v") for word in new_tweet.split()]) # Word Lemmatization
    
    return new_tweet

In [4]:
X = X.apply(preprocess)

print(X)

0       real reason sad attach people distant pay atte...
1                 biggest problem overthinking everything
2                        worst sadness sadness teach hide
3       make understand make anyone understand happen ...
4       think anyone really understand tire act okay a...
                              ...                        
8130    cardi b want trademark catchphrase okurr think...
8131    bet kellyanne george conway pretty disturb mak...
8132    fan always ask watch old stuff finally answer ...
8133    ray romano hilarious comedian kind soul rare n...
8134    mueller report may finish mine next week johnn...
Name: tweet, Length: 8135, dtype: object


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("TRAIN size:", len(X_train))
print("TEST size:", len(X_test))

TRAIN size: 6508
TEST size: 1627


In [6]:
w2v_size = 300
w2v_win = 7
w2v_epoch = 32
w2v_mincount = 10

document = [tweet.split() for tweet in X_train]
word2vec_model = gensim.models.word2vec.Word2Vec(vector_size=w2v_size,
                                                window=w2v_win,
                                                min_count=w2v_mincount,
                                                workers=8)

word2vec_model.build_vocab(document)

In [7]:
words = word2vec_model.wv.index_to_key
vocabulary_size = len(words)
print('Vocabulary_size ::: ',vocabulary_size)

Vocabulary_size :::  964


In [8]:
word2vec_model.train(document,total_examples=len(document),epochs=w2v_epoch)

(1185622, 1851840)

In [9]:
max_sequence_length = 300
vector_size = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
unique_tokens = tokenizer.word_index
tokens_size = len(unique_tokens)
print('No.of unique tokens === %s'%tokens_size)

No.of unique tokens === 8635


In [10]:
X_train_padded = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_padded,maxlen=max_sequence_length)

In [11]:
vector_matrix = np.zeros((tokens_size+1,w2v_size))

for word,i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        vector_matrix[i]=word2vec_model.wv[word]

print('vector matrix shape === ',vector_matrix.shape)

vector_matrix

vector matrix shape ===  (8636, 300)


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.07327162,  0.35163948, -0.04450747, ...,  0.01208324,
         0.06084003, -0.092077  ],
       [-0.41569623,  0.21939704, -0.33398464, ..., -0.43874168,
         0.56521469, -0.63016099],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [12]:
X_test_padded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_padded,maxlen=max_sequence_length)

In [35]:
from keras.layers.convolutional import MaxPooling1D
def build_lstm_model(optimizer, dropout_rate, n_neurons, cnn_activation, dense_activation):
    LSTM_model = Sequential(name='LSTM_model')
    LSTM_model.add(Embedding(tokens_size+1,w2v_size, weights=[vector_matrix],input_length=max_sequence_length,trainable=False))
    LSTM_model.add(Dropout(0.5))
    LSTM_model.add(Conv1D(filters=300, kernel_size=3, padding='same', activation='relu'))
    LSTM_model.add(MaxPooling1D(pool_size=2))
    LSTM_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    LSTM_model.add(Dense(1, activation='sigmoid'))
    LSTM_model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    return LSTM_model

In [36]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping

es_cb = EarlyStopping(monitor='val_loss', patience=20, mode='min', restore_best_weights=True)
classifier = KerasClassifier(build_lstm_model)

classifier.get_params().keys()

params = {
    'optimizer' : ['sgd', 'adam'],
    'dropout_rate' : [0.2, 0.3, 0.1],
    'n_neurons' : [300, 128, 64],
    'cnn_activation' : ['relu', 'elu'],
    'dense_activation' : ['relu', 'elu','sigmoid']
}

rand_search = RandomizedSearchCV(classifier, params, n_iter=10, cv=3, verbose=1, scoring='accuracy', return_train_score=True)
rand_search.fit(X_train_padded,Y_train,validation_split=0.1, epochs=100,
callbacks=es_cb)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100


C:\Users\Kerillos\AppData\Local\Temp\ipykernel_12752\1161767166.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_lstm_model)


122/122 [==============================] - 43s 320ms/step - loss: 0.3735 - accuracy: 0.8281 - val_loss: 0.3362 - val_accuracy: 0.8571
Epoch 2/100
122/122 [==============================] - 39s 316ms/step - loss: 0.3238 - accuracy: 0.8599 - val_loss: 0.2759 - val_accuracy: 0.8848
Epoch 3/100
122/122 [==============================] - 38s 316ms/step - loss: 0.3051 - accuracy: 0.8691 - val_loss: 0.3240 - val_accuracy: 0.8525
Epoch 4/100
122/122 [==============================] - 39s 317ms/step - loss: 0.2930 - accuracy: 0.8724 - val_loss: 0.2710 - val_accuracy: 0.8825
Epoch 5/100
122/122 [==============================] - 39s 317ms/step - loss: 0.2798 - accuracy: 0.8819 - val_loss: 0.2778 - val_accuracy: 0.8710
Epoch 6/100
122/122 [==============================] - 38s 314ms/step - loss: 0.2608 - accuracy: 0.8850 - val_loss: 0.2795 - val_accuracy: 0.8779
Epoch 7/100
122/122 [==============================] - 38s 314ms/step - loss: 0.2448 - accuracy: 0.8986 - val_loss: 0.2980 - val_accurac

Epoch 6/100
123/123 [==============================] - 76s 614ms/step - loss: 0.2460 - accuracy: 0.8988 - val_loss: 0.3635 - val_accuracy: 0.8571
Epoch 7/100
123/123 [==============================] - 77s 624ms/step - loss: 0.2402 - accuracy: 0.8999 - val_loss: 0.3988 - val_accuracy: 0.8456
Epoch 8/100
123/123 [==============================] - 78s 637ms/step - loss: 0.2207 - accuracy: 0.9109 - val_loss: 0.3947 - val_accuracy: 0.8341
Epoch 9/100
123/123 [==============================] - 76s 616ms/step - loss: 0.2090 - accuracy: 0.9114 - val_loss: 0.4044 - val_accuracy: 0.8618
Epoch 10/100
123/123 [==============================] - 77s 629ms/step - loss: 0.1854 - accuracy: 0.9186 - val_loss: 0.4082 - val_accuracy: 0.8594
Epoch 11/100
123/123 [==============================] - 76s 615ms/step - loss: 0.1692 - accuracy: 0.9265 - val_loss: 0.4484 - val_accuracy: 0.8618
Epoch 12/100
123/123 [==============================] - 75s 613ms/step - loss: 0.1768 - accuracy: 0.9278 - val_loss: 0.459

Epoch 39/100
122/122 [==============================] - 39s 323ms/step - loss: 0.3198 - accuracy: 0.8612 - val_loss: 0.3368 - val_accuracy: 0.8548
Epoch 40/100
122/122 [==============================] - 40s 329ms/step - loss: 0.3191 - accuracy: 0.8596 - val_loss: 0.3322 - val_accuracy: 0.8548
Epoch 41/100
122/122 [==============================] - 40s 326ms/step - loss: 0.3166 - accuracy: 0.8624 - val_loss: 0.3286 - val_accuracy: 0.8548
Epoch 42/100
122/122 [==============================] - 40s 325ms/step - loss: 0.3193 - accuracy: 0.8642 - val_loss: 0.3187 - val_accuracy: 0.8618
Epoch 43/100
122/122 [==============================] - 39s 319ms/step - loss: 0.3190 - accuracy: 0.8645 - val_loss: 0.3217 - val_accuracy: 0.8548
Epoch 44/100
122/122 [==============================] - 39s 319ms/step - loss: 0.3136 - accuracy: 0.8648 - val_loss: 0.3161 - val_accuracy: 0.8594
Epoch 45/100
122/122 [==============================] - 39s 317ms/step - loss: 0.3158 - accuracy: 0.8650 - val_loss: 0

...

Epoch 32/100
123/123 [==============================] - 76s 615ms/step - loss: 0.3308 - accuracy: 0.8528 - val_loss: 0.3321 - val_accuracy: 0.8410
Epoch 33/100
123/123 [==============================] - 76s 619ms/step - loss: 0.3334 - accuracy: 0.8538 - val_loss: 0.3753 - val_accuracy: 0.8018
Epoch 34/100
123/123 [==============================] - 74s 604ms/step - loss: 0.3328 - accuracy: 0.8533 - val_loss: 0.3155 - val_accuracy: 0.8756
Epoch 35/100
123/123 [==============================] - 75s 610ms/step - loss: 0.3281 - accuracy: 0.8561 - val_loss: 0.3134 - val_accuracy: 0.8756
Epoch 36/100
123/123 [==============================] - 75s 608ms/step - loss: 0.3300 - accuracy: 0.8553 - val_loss: 0.3170 - val_accuracy: 0.8687
Epoch 37/100
123/123 [==============================] - 75s 610ms/step - loss: 0.3296 - accuracy: 0.8540 - val_loss: 0.3038 - val_accuracy: 0.8779
Epoch 38/100
123/123 [==============================] - 75s 609ms/step - loss: 0.3266 - accuracy: 0.8558 - val_loss: 0

Epoch 42/100
123/123 [==============================] - 79s 643ms/step - loss: 0.3017 - accuracy: 0.8712 - val_loss: 0.3711 - val_accuracy: 0.8433
Epoch 43/100
123/123 [==============================] - 80s 649ms/step - loss: 0.3025 - accuracy: 0.8755 - val_loss: 0.3718 - val_accuracy: 0.8433
Epoch 44/100
123/123 [==============================] - 78s 637ms/step - loss: 0.3040 - accuracy: 0.8712 - val_loss: 0.3745 - val_accuracy: 0.8479
Epoch 45/100
123/123 [==============================] - 96s 785ms/step - loss: 0.3027 - accuracy: 0.8730 - val_loss: 0.3729 - val_accuracy: 0.8479
Epoch 46/100
123/123 [==============================] - 112s 915ms/step - loss: 0.3033 - accuracy: 0.8755 - val_loss: 0.3769 - val_accuracy: 0.8502
Epoch 47/100
123/123 [==============================] - 114s 926ms/step - loss: 0.3021 - accuracy: 0.8735 - val_loss: 0.3730 - val_accuracy: 0.8479
Epoch 48/100
123/123 [==============================] - 114s 925ms/step - loss: 0.2956 - accuracy: 0.8786 - val_loss

Epoch 81/100
122/122 [==============================] - 59s 483ms/step - loss: 0.2914 - accuracy: 0.8722 - val_loss: 0.2996 - val_accuracy: 0.8594
Epoch 82/100
122/122 [==============================] - 58s 477ms/step - loss: 0.2918 - accuracy: 0.8773 - val_loss: 0.3260 - val_accuracy: 0.8456
Epoch 83/100
122/122 [==============================] - 59s 480ms/step - loss: 0.2909 - accuracy: 0.8765 - val_loss: 0.3046 - val_accuracy: 0.8571
Epoch 84/100
122/122 [==============================] - 60s 490ms/step - loss: 0.2924 - accuracy: 0.8709 - val_loss: 0.3074 - val_accuracy: 0.8548
Epoch 85/100
122/122 [==============================] - 58s 477ms/step - loss: 0.2920 - accuracy: 0.8737 - val_loss: 0.3011 - val_accuracy: 0.8594
Epoch 86/100
122/122 [==============================] - 58s 477ms/step - loss: 0.2910 - accuracy: 0.8750 - val_loss: 0.2911 - val_accuracy: 0.8664
Epoch 87/100
122/122 [==============================] - 60s 488ms/step - loss: 0.2864 - accuracy: 0.8709 - val_loss: 0

Epoch 91/100
123/123 [==============================] - 77s 630ms/step - loss: 0.2900 - accuracy: 0.8725 - val_loss: 0.2781 - val_accuracy: 0.8802
Epoch 92/100
123/123 [==============================] - 78s 638ms/step - loss: 0.2888 - accuracy: 0.8768 - val_loss: 0.2691 - val_accuracy: 0.8733
Epoch 93/100
123/123 [==============================] - 79s 639ms/step - loss: 0.2908 - accuracy: 0.8709 - val_loss: 0.2723 - val_accuracy: 0.8756
Epoch 94/100
123/123 [==============================] - 80s 655ms/step - loss: 0.2911 - accuracy: 0.8753 - val_loss: 0.2702 - val_accuracy: 0.8664
Epoch 95/100
123/123 [==============================] - 76s 616ms/step - loss: 0.2857 - accuracy: 0.8796 - val_loss: 0.2735 - val_accuracy: 0.8802
Epoch 96/100
123/123 [==============================] - 76s 621ms/step - loss: 0.2871 - accuracy: 0.8781 - val_loss: 0.2698 - val_accuracy: 0.8756
Epoch 97/100
123/123 [==============================] - 77s 628ms/step - loss: 0.2872 - accuracy: 0.8778 - val_loss: 0

KeyboardInterrupt: 